#### **Census TopDown algorithm**
Abowd, J., Ashmead, R., Simson, G., Kifer, D., Leclerc, P., Machanavajjhala, A., & Sexton, W. (2019). Census topdown: Differentially private data, incremental schemas, and consistency with public knowledge. US Census Bureau.

In [1]:
from fractions import Fraction
import lib.discretegauss as discretegauss
import lib.cdp2adp as cdp2adp
# import math

def gauss_param(eps, delta, f):
    # convert to concentrated DP
    rho = cdp2adp.cdp_rho(eps, delta)
    rho_per_q = Fraction(rho) * f
    # compute noise variance parameter per query
    sigma2 = 2 ** 2 / (2 * rho_per_q)
    # actual variance, at most sigma2
    # var = discretegauss.variance(sigma2)
    # print(str(rho) + "-CDP implies (" + str(eps) + "," + str(delta) + ")-DP")
    # print("standard deviation for each count = " + str(math.sqrt(var)))
    return sigma2

##### **County level**

Initialize county-level noisy queries 

In [6]:
import pandas as pd
import lib.discretegauss as discretegauss

filename_sf1 = 'data/franklin/microdata/franklin_hist.csv'
hist_sf1 = pd.read_csv(filename_sf1)

# setup parameters
eps = 17.14
delta = 1e-10
n1, n2, n3, n4 = 8, 2, 2, 63        # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)
f2, f3, f4, f6, f7, f8, f9, f10, f11 = 447/4099*10/4097, 447/4099*10/4097, 447/4099*10/4097, 447/4099*10/4097, 447/4099*28/4097, 447/4099*28/4097, 447/4099*10/4097, 447/4099*101/4097, 447/4099*754/4097
m1, m2, m3, m4, m6, m7, m8, m9, m10, m11 = [], [], [], [], [], [], [], [], [], []
m1_dp, m2_dp, m3_dp, m4_dp, m6_dp, m7_dp, m8_dp, m9_dp, m10_dp, m11_dp = [], [], [], [], [], [], [], [], [], []

## county-level $H^0$
hist_cou = hist_sf1.sum(axis = 0).to_frame().T
hist_cou = hist_cou.drop(['GEOID10'], axis=1)

## noisy answers to workload queries
## Q1: TOTAL (1 cell) [invariant]
m1_dp = m1 = hist_cou.sum(axis = 1).values

## Q2: CENRACE (63 cells)
for x in range(n4):     # race
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in hist_cou.columns if x in col[6:8]]
    m2.append(hist_cou[col_names].sum(axis=1).values[0])
sigma2_2 = gauss_param(eps, delta, f2)
m2_dp = [i + discretegauss.sample_dgauss(sigma2_2) for i in m2]

## Q3: HISPANIC (2 cells)
for x in range(n3):     # hispanic
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in hist_cou.columns if x in col[4:6]]
    m3.append(hist_cou[col_names].sum(axis=1).values[0])
sigma2_3 = gauss_param(eps, delta, f3)
m3_dp = [i + discretegauss.sample_dgauss(sigma2_3) for i in m3]

## Q4: VOTINGAGE (2 cells)
for x in range(n2):     # voting age
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in hist_cou.columns if x in col[2:4]]
    m4.append(hist_cou[col_names].sum(axis=1).values[0])
sigma2_4 = gauss_param(eps, delta, f4)
m4_dp = [i + discretegauss.sample_dgauss(sigma2_4) for i in m4]

## Q6: HHGQ (8 cells)
for x in range(n1):  # hhgq
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in hist_cou.columns if x in col[0:2]]
    m6.append(hist_cou[col_names].sum(axis=1).values[0])
sigma2_6 = gauss_param(eps, delta, f6)
m6_dp = [i + discretegauss.sample_dgauss(sigma2_6) for i in m6]

## Q7: HISPANIC*CENRACE (126 cells)
for x in range(n3):     # hispanic
    x = '{number:0{width}d}'.format(width=2, number=x)
    for y in range(n4):     # race
        y = '{number:0{width}d}'.format(width=2, number=y)
        col_names = [col for col in hist_cou.columns if x in col[4:6] and y in col[6:8]]
        m7.append(hist_cou[col_names].sum(axis=1).values[0])
sigma2_7 = gauss_param(eps, delta, f7)
m7_dp = [i + discretegauss.sample_dgauss(sigma2_7) for i in m7]

## Q8: VOTINGAGE*CENRACE (126 cells)
for x in range(n2):     # voting age
    x = '{number:0{width}d}'.format(width=2, number=x)
    for y in range(n4):     # race
        y = '{number:0{width}d}'.format(width=2, number=y)
        col_names = [col for col in hist_cou.columns if x in col[2:4] and y in col[6:8]]
        m8.append(hist_cou[col_names].sum(axis=1).values[0])
sigma2_8 = gauss_param(eps, delta, f8)
m8_dp = [i + discretegauss.sample_dgauss(sigma2_8) for i in m8]

## Q9: VOTINGAGE*HISPANIC (4 cells)
for x in range(n2):     # voting age
    x = '{number:0{width}d}'.format(width=2, number=x)
    for y in range(n3):     # hispanic
        y = '{number:0{width}d}'.format(width=2, number=y)
        col_names = [col for col in hist_cou.columns if x in col[2:4] and y in col[4:6]]
        m9.append(hist_cou[col_names].sum(axis=1).values[0])
sigma2_9 = gauss_param(eps, delta, f9)
m9_dp = [i + discretegauss.sample_dgauss(sigma2_9) for i in m9]

## Q10: VOTINGAGE*HISPANIC*CENRACE (252 cells)
for x in range(n2):     # voting age
    x = '{number:0{width}d}'.format(width=2, number=x)
    for y in range(n3):     # hispanic
        y = '{number:0{width}d}'.format(width=2, number=y)
        for z in range(n4):     # race
            z = '{number:0{width}d}'.format(width=2, number=z)
            col_names = [col for col in hist_cou.columns if x in col[2:4] and y in col[4:6] and z in col[6:8]]
            m10.append(hist_cou[col_names].sum(axis=1).values[0])
sigma2_10 = gauss_param(eps, delta, f10)
m10_dp = [i + discretegauss.sample_dgauss(sigma2_10) for i in m10]

## Q11: HHGQ*VOTINGAGE*HISPANIC*CENRACE (2,016 cells)
m11 = hist_cou.to_numpy()[0]
sigma2_11 = gauss_param(eps, delta, f11)
m11_dp = [i + discretegauss.sample_dgauss(sigma2_11) for i in m11]

print(len(m1_dp), len(m2_dp), len(m3_dp), len(m4_dp), len(m6_dp), len(m7_dp), len(m8_dp), len(m9_dp), len(m10_dp), len(m11_dp))
hist_cou

1 63 2 2 8 126 126 4 252 2016


,00000000,00000001,00000002,00000003,00000004,00000005,00000006,00000007,00000008,00000009,...,07010153,07010154,07010155,07010156,07010157,07010158,07010159,07010160,07010161,07010162
0,156629,74881,478,9999,266,1396,4018,443,1099,27,...,0,0,0,0,0,0,0,0,0,0


Optimize county-level histogram

In [7]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, QuadExpr, quicksum
import torch
import csv

# set up parameters
n1, n2, n3, n4 = 8, 2, 2, 63
N = n1 * n2 * n3 * n4         # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)

with open('data/franklin/microdata/franklin_hist_dp_cou.csv', 'w', newline='') as f:
    wr = csv.writer(f)
    # set up column names
    hist_names = np.empty((n1, n2, n3, n4), dtype="U8")
    for k1 in range(n1):
        for k2 in range(n2):
            for k3 in range(n3):
                for k4 in range(n4):
                    hist_names[k1, k2, k3, k4] = str(k1).zfill(2) + str(k2).zfill(2) + str(k3).zfill(2) + str(k4).zfill(2)
    wr.writerow(hist_names.flatten())

    A = torch.tensor(range(N))
    A = A.reshape([n1, n2, n3, n4])

    # initialize model
    m = Model('td')
    m.Params.timelimit = 100.0
    # m.Params.LogToConsole = 0
    
    # add objective function
    obj = QuadExpr()

    # add variables and constraints
    h = {}      ## detailed histogram (decision vairable)
    for i in range(N):
        h[i] = m.addVar(obj=0, vtype=GRB.INTEGER, name="h_%d"%(i))
    m.addConstr(quicksum(h[i] for i in range(N)) == m1[0])
    m.update()

    ## Q2: CENRACE (63 cells)
    res2, col_idx = {}, 0
    for x in range(n4):     # race   
        hist_idx = torch.flatten(A[:, :, :, x]).tolist()
        res2[col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res2_%d"%(col_idx))
        obj += res2[col_idx] * res2[col_idx]
        m.addConstr(res2[col_idx] == m2_dp[col_idx] - quicksum(h[i] for i in hist_idx))
        m.update()
        col_idx += 1
    
    ## Q3: HISPANIC (2 cells)
    res3, col_idx = {}, 0
    for x in range(n3):     # hispanic   
        hist_idx = torch.flatten(A[:, :, x, :]).tolist()
        res3[col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res3_%d"%(col_idx))
        obj += res3[col_idx] * res3[col_idx]
        m.addConstr(res3[col_idx] == m3_dp[col_idx] - quicksum(h[i] for i in hist_idx))
        m.update()
        col_idx += 1
        
    ## Q4: VOTINGAGE (2 cells)
    res4, col_idx = {}, 0
    for x in range(n2):     # voting age  
        hist_idx = torch.flatten(A[:, x, :, :]).tolist()
        res4[col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res4_%d"%(col_idx))
        obj += res4[col_idx] * res4[col_idx]
        m.addConstr(res4[col_idx] == m4_dp[col_idx] - quicksum(h[i] for i in hist_idx))
        m.update()
        col_idx += 1
    
    ## Q6: HHGQ (8 cells)
    res6, col_idx = {}, 0
    for x in range(n1):     # hhgq       
        hist_idx = torch.flatten(A[x, :, :, :]).tolist()
        res6[col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res6_%d"%(col_idx))
        obj += res6[col_idx] * res6[col_idx]
        m.addConstr(res6[col_idx] == m6_dp[col_idx] - quicksum(h[i] for i in hist_idx))
        m.update()
        col_idx += 1
    
    ## Q7: HISPANIC*CENRACE (126 cells)
    res7, col_idx = {}, 0
    for x in range(n3):     # hispanic
        for y in range(n4):     # race       
            hist_idx = torch.flatten(A[:, :, x, y]).tolist()
            res7[col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res7_%d"%(col_idx))
            obj += res7[col_idx] * res7[col_idx]
            m.addConstr(res7[col_idx] == m7_dp[col_idx] - quicksum(h[i] for i in hist_idx))
            m.update()
            col_idx += 1

    ## Q8: VOTINGAGE*CENRACE (126 cells)
    res8, col_idx = {}, 0
    for x in range(n2):     # voting age
        for y in range(n4):     # race       
            hist_idx = torch.flatten(A[:, x, :, y]).tolist()
            res8[col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res8_%d"%(col_idx))
            obj += res8[col_idx] * res8[col_idx]
            m.addConstr(res8[col_idx] == m8_dp[col_idx] - quicksum(h[i] for i in hist_idx))
            m.update()
            col_idx += 1
    
    ## Q9: VOTINGAGE*HISPANIC (4 cells)
    res9, col_idx = {}, 0
    for x in range(n2):     # voting age
        for y in range(n3):     # hispanic       
            hist_idx = torch.flatten(A[:, x, y, :]).tolist()
            res9[col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res9_%d"%(col_idx))
            obj += res9[col_idx] * res9[col_idx]
            m.addConstr(res9[col_idx] == m9_dp[col_idx] - quicksum(h[i] for i in hist_idx))
            m.update()
            col_idx += 1 

    ## Q10: VOTINGAGE*HISPANIC*CENRACE (252 cells)
    res10, col_idx = {}, 0
    for x in range(n2):     # voting age
        for y in range(n3):     # hispanic
            for z in range(n4):     # race        
                hist_idx = torch.flatten(A[:, x, y, z]).tolist()
                res10[col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res10_%d"%(col_idx))
                obj += res10[col_idx] * res10[col_idx]
                m.addConstr(res10[col_idx] == m10_dp[col_idx] - quicksum(h[i] for i in hist_idx))
                m.update()
                col_idx += 1 

    ## Q11: HHGQ*VOTINGAGE*HISPANIC*CENRACE (2,016 cells)
    res11 = {}
    for i in range(N):
        res11[i] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res11_%d"%(i))
        obj += res11[i] * res11[i]
        m.addConstr(res11[i] == m11_dp[i] - h[i])                                
    m.update()
    
    m.setObjective(obj, GRB.MINIMIZE)
    m.optimize()

    ## write histogram values
    var_values = [var.X for var in m.getVars() if 'h' == str(var.VarName[0])]
    wr.writerow(var_values)

    obj = m.getObjective().getValue()
    print(obj)

Changed value of parameter timelimit to 100.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 2600 rows, 4615 columns and 22759 nonzeros
Model fingerprint: 0xa6127598
Model has 2599 quadratic objective terms
Variable types: 0 continuous, 4615 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 1.202805e+13
Presolve removed 2016 rows and 2016 columns
Presolve time: 0.07s
Presolved: 584 rows, 2599 columns, 18727 nonzeros
Presolved model has 2599 quadratic objective terms
Variable types: 0 continuous, 2599 integer (0 binary)

Root relaxation: objective 5.529938e+05, 1679 iterations, 0.58 seconds

    Nodes    |    Current Node    |     Objective 

Compare noises in race query between PPMF and my data

In [21]:
# ppmf
import pandas as pd

filename_nhgis = 'data/nhgis/nhgis_ppdd_20210608_county.csv'
data_nhgis = pd.read_csv(filename_nhgis, encoding = "ISO-8859-1")
data_nhgis["STATE"] = data_nhgis["gisjoin"].str.slice(1, 3)
data_nhgis['COUNTY'] = data_nhgis["gisjoin"].str.slice(4, 7)
data_nhgis = data_nhgis[(data_nhgis["STATE"] == '39') & (data_nhgis["COUNTY"] == '049')]
data_nhgis = data_nhgis[['H72003_dp', 'H72004_dp', 'H72005_dp', 'H72006_dp', 'H72007_dp', 'H72008_dp', 'H72009_dp', 
                         'H72003_sf', 'H72004_sf', 'H72005_sf', 'H72006_sf', 'H72007_sf', 'H72008_sf', 'H72009_sf']]

hist1_nhgis = data_nhgis[['H72003_sf', 'H72004_sf', 'H72005_sf', 'H72006_sf', 'H72007_sf', 'H72008_sf', 'H72009_sf']]
hist1_nhgis = data_nhgis.rename(columns={'H72003_sf': '1', 'H72004_sf': '2', 'H72005_sf': '3', 'H72006_sf': '4', 'H72007_sf': '5', 'H72008_sf': '6', 
                                          'H72009_sf': '7'})
hist2_nhgis = data_nhgis[['H72003_dp', 'H72004_dp', 'H72005_dp', 'H72006_dp', 'H72007_dp', 'H72008_dp', 'H72009_dp']]
hist2_nhgis = data_nhgis.rename(columns={'H72003_dp': '1', 'H72004_dp': '2', 'H72005_dp': '3', 'H72006_dp': '4', 'H72007_dp': '5', 'H72008_dp': '6', 
                                          'H72009_dp': '7'})

noise = abs(hist2_nhgis - hist1_nhgis).mean(axis=1).values[0]
noise

42.714285714285715

In [13]:
# my data
import pandas as pd
import numpy as np

n4 = 63
# original hist
filename_hist = 'data/franklin/microdata/franklin_hist.csv'
hist_cou = pd.read_csv(filename_hist)
hist_cou = hist_cou.sum(axis = 0).to_frame().T
hist_cou = hist_cou.drop(['GEOID10'], axis=1)

m2 = []
## Q2: CENRACE (63 cells)
for x in range(n4):     # race
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in hist_cou.columns if x in col[6:8]]
    m2.append(hist_cou[col_names].sum(axis=1).values[0])

# my noisy hist
filename_sf1 = 'data/franklin/microdata/franklin_hist_dp_cou.csv'
sf1 = pd.read_csv(filename_sf1)

m2_noisy = []
## Q2: CENRACE (63 cells)
for x in range(n4):     # race
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in sf1.columns if x in col[6:8]]
    m2_noisy.append(sf1[col_names].sum(axis=1).values[0])

noise = np.absolute(np.subtract(np.array(m2_noisy), np.array(m2)))

np.mean(noise)

14.793650793650794

##### **Tract level**

Initialize tract-level noisy queries

In [2]:
import pandas as pd
import numpy as np
import lib.discretegauss as discretegauss

filename_sf1 = 'data/franklin/microdata/franklin_hist.csv'
hist_sf1 = pd.read_csv(filename_sf1)

# setup parameters
eps = 17.14
delta = 1e-10
n1, n2, n3, n4 = 8, 2, 2, 63        # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)
f1, f2, f3, f4, f6, f7, f8, f9, f10, f11 = 687/4099*1567/4102, 687/4099*4/2051, 687/4099*5/4102, 687/4099*5/4102, 687/4099*5/4102, 687/4099*1933/4102, 687/4099*10/2051, 687/4099*5/4102, 687/4099*67/4102, 687/4099*241/2051
m1, m2, m3, m4, m6, m7, m8, m9, m10, m11 = [], [], [], [], [], [], [], [], [], []
m1_dp, m2_dp, m3_dp, m4_dp, m6_dp, m7_dp, m8_dp, m9_dp, m10_dp, m11_dp = [], [], [], [], [], [], [], [], [], []

## county-level $H^0$
hist_sf1['TRACT'] = hist_sf1['GEOID10'].astype(str).str[:11]
col_names = hist_sf1.columns.to_numpy()
col_names = np.delete(col_names, [0, -1])
hist_tr = hist_sf1.groupby('TRACT').sum()[col_names]
hist_tr["TRACT"] = hist_tr.index.map(str)
hist_tr.index.name = None
hist_tr = hist_tr.reset_index(drop=True)

## noisy answers to workload queries
## Q1: TOTAL (1 cell) [invariant]
m1 = hist_tr.sum(axis = 1).values
sigma2_1 = gauss_param(eps, delta, f1)
m1_dp = np.array([i + discretegauss.sample_dgauss(sigma2_1) for i in m1])

## Q2: CENRACE (63 cells)
for x in range(n4):     # race
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in hist_tr.columns if x in col[6:8]]
    m2.append(hist_tr[col_names].sum(axis=1).values)
sigma2_2 = gauss_param(eps, delta, f2)
m2_dp = np.array([[i + discretegauss.sample_dgauss(sigma2_2) for i in row] for row in m2]).T

## Q3: HISPANIC (2 cells)
for x in range(n3):     # hispanic
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in hist_tr.columns if x in col[4:6]]
    m3.append(hist_tr[col_names].sum(axis=1).values)
sigma2_3 = gauss_param(eps, delta, f3)
m3_dp = np.array([[i + discretegauss.sample_dgauss(sigma2_3) for i in row] for row in m3]).T

## Q4: VOTINGAGE (2 cells)
for x in range(n2):     # voting age
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in hist_tr.columns if x in col[2:4]]
    m4.append(hist_tr[col_names].sum(axis=1).values)
sigma2_4 = gauss_param(eps, delta, f4)
m4_dp = np.array([[i + discretegauss.sample_dgauss(sigma2_4) for i in row] for row in m4]).T

## Q6: HHGQ (8 cells)
for x in range(n1):  # hhgq
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in hist_tr.columns if x in col[0:2]]
    m6.append(hist_tr[col_names].sum(axis=1).values)
sigma2_6 = gauss_param(eps, delta, f6)
m6_dp = np.array([[i + discretegauss.sample_dgauss(sigma2_6) for i in row] for row in m6]).T

## Q7: HISPANIC*CENRACE (126 cells)
for x in range(n3):     # hispanic
    x = '{number:0{width}d}'.format(width=2, number=x)
    for y in range(n4):     # race
        y = '{number:0{width}d}'.format(width=2, number=y)
        col_names = [col for col in hist_tr.columns if x in col[4:6] and y in col[6:8]]
        m7.append(hist_tr[col_names].sum(axis=1).values)
sigma2_7 = gauss_param(eps, delta, f7)
m7_dp = np.array([[i + discretegauss.sample_dgauss(sigma2_7) for i in row] for row in m7]).T

## Q8: VOTINGAGE*CENRACE (126 cells)
for x in range(n2):     # voting age
    x = '{number:0{width}d}'.format(width=2, number=x)
    for y in range(n4):     # race
        y = '{number:0{width}d}'.format(width=2, number=y)
        col_names = [col for col in hist_tr.columns if x in col[2:4] and y in col[6:8]]
        m8.append(hist_tr[col_names].sum(axis=1).values)
sigma2_8 = gauss_param(eps, delta, f8)
m8_dp = np.array([[i + discretegauss.sample_dgauss(sigma2_8) for i in row] for row in m8]).T

## Q9: VOTINGAGE*HISPANIC (4 cells)
for x in range(n2):     # voting age
    x = '{number:0{width}d}'.format(width=2, number=x)
    for y in range(n3):     # hispanic
        y = '{number:0{width}d}'.format(width=2, number=y)
        col_names = [col for col in hist_tr.columns if x in col[2:4] and y in col[4:6]]
        m9.append(hist_tr[col_names].sum(axis=1).values)
sigma2_9 = gauss_param(eps, delta, f9)
m9_dp = np.array([[i + discretegauss.sample_dgauss(sigma2_9) for i in row] for row in m9]).T

## Q10: VOTINGAGE*HISPANIC*CENRACE (252 cells)
for x in range(n2):     # voting age
    x = '{number:0{width}d}'.format(width=2, number=x)
    for y in range(n3):     # hispanic
        y = '{number:0{width}d}'.format(width=2, number=y)
        for z in range(n4):     # race
            z = '{number:0{width}d}'.format(width=2, number=z)
            col_names = [col for col in hist_tr.columns if x in col[2:4] and y in col[4:6] and z in col[6:8]]
            m10.append(hist_tr[col_names].sum(axis=1).values)
sigma2_10 = gauss_param(eps, delta, f10)
m10_dp = np.array([[i + discretegauss.sample_dgauss(sigma2_10) for i in row] for row in m10]).T

## Q11: HHGQ*VOTINGAGE*HISPANIC*CENRACE (2,016 cells)
m11 = hist_tr.drop(['TRACT'], axis=1).to_numpy()
sigma2_11 = gauss_param(eps, delta, f11)
m11_dp = np.array([[i + discretegauss.sample_dgauss(sigma2_11) for i in row] for row in m11])

print(m1_dp.shape, m2_dp.shape, m3_dp.shape, m4_dp.shape, m6_dp.shape, m7_dp.shape, m8_dp.shape, m9_dp.shape, m10_dp.shape, m11_dp.shape)
hist_tr

(284,) (284, 63) (284, 2) (284, 2) (284, 8) (284, 126) (284, 126) (284, 4) (284, 252) (284, 2016)


,00000000,00000001,00000002,00000003,00000004,00000005,00000006,00000007,00000008,00000009,...,07010154,07010155,07010156,07010157,07010158,07010159,07010160,07010161,07010162,TRACT
0,438,6,0,7,0,4,5,1,0,0,...,0,0,0,0,0,0,0,0,0,39049000110
1,467,4,0,11,0,2,2,0,2,0,...,0,0,0,0,0,0,0,0,0,39049000120
2,430,11,0,6,0,4,4,1,4,0,...,0,0,0,0,0,0,0,0,0,39049000210
3,653,3,1,12,0,6,7,0,6,0,...,0,0,0,0,0,0,0,0,0,39049000220
4,313,205,0,17,0,2,19,0,4,0,...,0,0,0,0,0,0,0,0,0,39049000310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,1821,35,1,409,0,2,5,2,22,0,...,0,0,0,0,0,0,0,0,0,39049010500
280,1408,36,0,146,0,3,12,0,15,0,...,0,0,0,0,0,0,0,0,0,39049010601
281,1585,45,3,139,0,4,7,2,9,0,...,0,0,0,0,0,0,0,0,0,39049010602
282,83,11,0,4,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,39049010700


Optimize tract-level histogram

In [4]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, QuadExpr, quicksum
import torch
import csv

# set up parameters
n1, n2, n3, n4 = 8, 2, 2, 63
N = n1 * n2 * n3 * n4         # number of attribute combinations: HHGQ (8) ∗ VOTINGAGE (2) ∗ HISPANIC (2) ∗ RACE (63)
M = len(hist_tr)

# parent 
filename_cou = 'data/franklin/microdata/franklin_hist_dp_cou.csv'
hist_cou = pd.read_csv(filename_cou)

with open('data/franklin/microdata/franklin_hist_dp_tr.csv', 'w', newline='') as f:
    wr = csv.writer(f)
    # set up column names
    hist_names = np.empty((n1, n2, n3, n4), dtype="U8")
    for k1 in range(n1):
        for k2 in range(n2):
            for k3 in range(n3):
                for k4 in range(n4):
                    hist_names[k1, k2, k3, k4] = str(k1).zfill(2) + str(k2).zfill(2) + str(k3).zfill(2) + str(k4).zfill(2)
    wr.writerow(hist_names.flatten())

    A = torch.tensor(range(N))
    A = A.reshape([n1, n2, n3, n4])

    # initialize model
    m = Model('td')
    # m.Params.timelimit = 2000.0
    # m.Params.LogToConsole = 0
    
    # add objective function
    obj = QuadExpr()

    # add variables and constraints
    h = {}      ## detailed histogram (decision vairable)
    for i in range(M):
        for j in range(N):
            h[i, j] = m.addVar(obj=0, vtype=GRB.INTEGER, name="h_%d_%d"%(i,j))
    m.update()

    ## Q1: TOTAL (1 cell)
    res1 = {}
    for i in range(M):  
        res1[i] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res1_%d"%(i))
        obj += res1[i] * res1[i]
        m.addConstr(res1[i] == m1_dp[i] - quicksum(h[i, j] for j in range(N)))
    m.addConstr(quicksum(quicksum(h[i, j] for j in range(N)) for i in range(M)) == hist_cou.sum(axis = 1).values[0])
    m.update()

    ## Q2: CENRACE (63 cells)
    res2 = {}
    for i in range(M):
        col_idx = 0
        for x in range(n4):     # race   
            hist_idx = torch.flatten(A[:, :, :, x]).tolist()
            res2[i, col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res2_%d_%d"%(i, col_idx))
            obj += res2[i, col_idx] * res2[i, col_idx]
            m.addConstr(res2[i, col_idx] == m2_dp[i, col_idx] - quicksum(h[i, j] for j in hist_idx))
            col_idx += 1
    for x in range(n4):     # race   
        hist_idx = torch.flatten(A[:, :, :, x]).tolist()
        m.addConstr(quicksum(quicksum(h[i, j] for j in hist_idx) for i in range(M)) == hist_cou.iloc[:, hist_idx].sum(axis = 1).values[0])
    m.update()

    ## Q3: HISPANIC (2 cells)
    res3 = {}
    for i in range(M):
        col_idx = 0
        for x in range(n3):     # hispanic   
            hist_idx = torch.flatten(A[:, :, x, :]).tolist()
            res3[i, col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res3_%d_%d"%(i, col_idx))
            obj += res3[i, col_idx] * res3[i, col_idx]
            m.addConstr(res3[i, col_idx] == m3_dp[i, col_idx] - quicksum(h[i, j] for j in hist_idx))
            col_idx += 1
    for x in range(n3):     # hispanic   
        hist_idx = torch.flatten(A[:, :, x, :]).tolist()
        m.addConstr(quicksum(quicksum(h[i, j] for j in hist_idx) for i in range(M)) == hist_cou.iloc[:, hist_idx].sum(axis = 1).values[0])
    m.update()

    ## Q4: VOTINGAGE (2 cells)
    res4 = {}
    for i in range(M):
        col_idx = 0
        for x in range(n2):     # voting age  
            hist_idx = torch.flatten(A[:, x, :, :]).tolist()
            res4[i, col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res4_%d_%d"%(i, col_idx))
            obj += res4[i, col_idx] * res4[i, col_idx]
            m.addConstr(res4[i, col_idx] == m4_dp[i, col_idx] - quicksum(h[i, j] for j in hist_idx))
            col_idx += 1
    for x in range(n2):     # voting age   
        hist_idx = torch.flatten(A[:, x, :, :]).tolist()
        m.addConstr(quicksum(quicksum(h[i, j] for j in hist_idx) for i in range(M)) == hist_cou.iloc[:, hist_idx].sum(axis = 1).values[0])
    m.update()

    ## Q6: HHGQ (8 cells)
    res6 = {}
    for i in range(M):
        col_idx = 0
        for x in range(n1):     # hhgq       
            hist_idx = torch.flatten(A[x, :, :, :]).tolist()
            res6[i, col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res6_%d_%d"%(i, col_idx))
            obj += res6[i, col_idx] * res6[i, col_idx]
            m.addConstr(res6[i, col_idx] == m6_dp[i, col_idx] - quicksum(h[i, j] for j in hist_idx))
            col_idx += 1
    for x in range(n1):     # voting age   
        hist_idx = torch.flatten(A[x, :, :, :]).tolist()
        m.addConstr(quicksum(quicksum(h[i, j] for j in hist_idx) for i in range(M)) == hist_cou.iloc[:, hist_idx].sum(axis = 1).values[0])
    m.update()

    ## Q7: HISPANIC*CENRACE (126 cells)
    res7 = {}
    for i in range(M):
        col_idx = 0
        for x in range(n3):     # hispanic
            for y in range(n4):     # hispanic       
                hist_idx = torch.flatten(A[:, :, x, y]).tolist()
                res7[i, col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res7_%d_%d"%(i, col_idx))
                obj += res7[i, col_idx] * res7[i, col_idx]
                m.addConstr(res7[i, col_idx] == m7_dp[i, col_idx] - quicksum(h[i, j] for j in hist_idx))                
                col_idx += 1
    for x in range(n3):     # hispanic
        for y in range(n4):     # race  
            hist_idx = torch.flatten(A[:, :, x, y]).tolist()
            m.addConstr(quicksum(quicksum(h[i, j] for j in hist_idx) for i in range(M)) == hist_cou.iloc[:, hist_idx].sum(axis = 1).values[0])
    m.update()

    ## Q8: VOTINGAGE*CENRACE (126 cells)
    res8 = {}
    for i in range(M):
        col_idx = 0
        for x in range(n2):     # voting age
            for y in range(n4):     # race       
                hist_idx = torch.flatten(A[:, x, :, y]).tolist()
                res8[i, col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res8_%d_%d"%(i, col_idx))
                obj += res8[i, col_idx] * res8[i, col_idx]
                m.addConstr(res8[i, col_idx] == m8_dp[i, col_idx] - quicksum(h[i, j] for j in hist_idx))                
                col_idx += 1
    for x in range(n2):     # voting age
        for y in range(n4):     # race    
            hist_idx = torch.flatten(A[:, x, :, y]).tolist()
            m.addConstr(quicksum(quicksum(h[i, j] for j in hist_idx) for i in range(M)) == hist_cou.iloc[:, hist_idx].sum(axis = 1).values[0])
    m.update()

    ## Q9: VOTINGAGE*HISPANIC (4 cells)
    res9 = {}
    for i in range(M):
        col_idx = 0
        for x in range(n2):     # voting age
            for y in range(n3):     # hispanic       
                hist_idx = torch.flatten(A[:, x, y, :]).tolist()
                res9[i, col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res9_%d_%d"%(i, col_idx))
                obj += res9[i, col_idx] * res9[i, col_idx]
                m.addConstr(res9[i, col_idx] == m9_dp[i, col_idx] - quicksum(h[i, j] for j in hist_idx))                
                col_idx += 1 
    for x in range(n2):     # voting age
        for y in range(n3):     # hispanic  
            hist_idx = torch.flatten(A[:, x, y, :]).tolist()
            m.addConstr(quicksum(quicksum(h[i, j] for j in hist_idx) for i in range(M)) == hist_cou.iloc[:, hist_idx].sum(axis = 1).values[0])
    m.update()

    ## Q10: VOTINGAGE*HISPANIC*CENRACE (252 cells)
    res10 = {}
    for i in range(M):
        col_idx = 0
        for x in range(n2):     # voting age
            for y in range(n3):     # hispanic
                for z in range(n4):     # race        
                    hist_idx = torch.flatten(A[:, x, y, z]).tolist()
                    res10[i, col_idx] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res10_%d_%d"%(i, col_idx))
                    obj += res10[i, col_idx] * res10[i, col_idx]
                    m.addConstr(res10[i, col_idx] == m10_dp[i, col_idx] - quicksum(h[i, j] for j in hist_idx))
                    col_idx += 1
    for x in range(n2):     # voting age
        for y in range(n3):     # hispanic
            for z in range(n4):     # race  
                hist_idx = torch.flatten(A[:, x, y, z]).tolist()
                m.addConstr(quicksum(quicksum(h[i, j] for j in hist_idx) for i in range(M)) == hist_cou.iloc[:, hist_idx].sum(axis = 1).values[0])
    m.update()

    ## Q11: HHGQ*VOTINGAGE*HISPANIC*CENRACE (2,016 cells)
    res11 = {}
    for i in range(M):
        col_idx = 0
        for j in range(N):
            res11[i, j] = m.addVar(obj=0, lb=-GRB.INFINITY, vtype=GRB.INTEGER, name="res11_%d_%d"%(i, j))
            obj += res11[i, j] * res11[i, j]
            m.addConstr(res11[i, j] == m11_dp[i, j] - h[i, j])                                
    for j in range(N):
        m.addConstr(quicksum(h[i, j] for i in range(M)) == hist_cou.iloc[:, j].values[0])
    m.update()
    
    m.setObjective(obj, GRB.MINIMIZE)
    m.optimize()

    ## write histogram values
    for i in range(M):
        row = [hist_tr.iloc[i]['TRACT']]
        var_values = [var.X for var in m.getVars() if 'h_%d_' % i in str(var.VarName)]
        row.extend(var_values)
        wr.writerow(row)

    ## print objective values
    obj = m.getObjective().getValue()
    print(obj)
    print(m)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 741000 rows, 1310944 columns and 12189280 nonzeros
Model fingerprint: 0xade7e3ae
Model has 738400 quadratic objective terms
Variable types: 0 continuous, 1310944 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 383449 rows and 884376 columns (presolve time = 5s) ...
Presolve removed 650783 rows and 1151392 columns (presolve time = 11s) ...
Presolve removed 650783 rows and 1151392 columns
Presolve time: 12.32s
Presolved: 90217 rows, 159552 columns, 749080 nonzeros
Presolved model has 159063 quadratic objective terms
Variable types: 0 continuous, 159552 integer (489 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
 

Compare noises in race query between PPMF and my data

In [14]:
# ppmf
import pandas as pd

filename_nhgis = 'data/nhgis/nhgis_ppdd_20210608_tract.csv'
data_nhgis = pd.read_csv(filename_nhgis, encoding = "ISO-8859-1")
data_nhgis["STATE"] = data_nhgis["gisjoin"].str.slice(1, 3)
data_nhgis['COUNTY'] = data_nhgis["gisjoin"].str.slice(4, 7)
data_nhgis['TRACT'] = data_nhgis["gisjoin"].str.slice(8, 14)
data_nhgis = data_nhgis[(data_nhgis["STATE"] == '39') & (data_nhgis["COUNTY"] == '049')]
data_nhgis = data_nhgis[['H72003_dp', 'H72004_dp', 'H72005_dp', 'H72006_dp', 'H72007_dp', 'H72008_dp', 'H72009_dp', 
                         'H72003_sf', 'H72004_sf', 'H72005_sf', 'H72006_sf', 'H72007_sf', 'H72008_sf', 'H72009_sf']]

hist1_nhgis = data_nhgis[['H72003_sf', 'H72004_sf', 'H72005_sf', 'H72006_sf', 'H72007_sf', 'H72008_sf', 'H72009_sf']]
hist1_nhgis = data_nhgis.rename(columns={'H72003_sf': '1', 'H72004_sf': '2', 'H72005_sf': '3', 'H72006_sf': '4', 'H72007_sf': '5', 'H72008_sf': '6', 
                                          'H72009_sf': '7'})
hist2_nhgis = data_nhgis[['H72003_dp', 'H72004_dp', 'H72005_dp', 'H72006_dp', 'H72007_dp', 'H72008_dp', 'H72009_dp']]
hist2_nhgis = data_nhgis.rename(columns={'H72003_dp': '1', 'H72004_dp': '2', 'H72005_dp': '3', 'H72006_dp': '4', 'H72007_dp': '5', 'H72008_dp': '6', 
                                          'H72009_dp': '7'})

noise = abs(hist2_nhgis - hist1_nhgis).mean(axis=1).values[0]
noise

1.8571428571428572

In [12]:
# my data
import pandas as pd
import numpy as np

n4 = 63
# original hist
filename_hist = 'data/franklin/microdata/franklin_hist.csv'
hist_tr = pd.read_csv(filename_hist)
hist_tr['TRACT'] = hist_tr['GEOID10'].astype(str).str[:11]
col_names = hist_tr.columns.to_numpy()
col_names = np.delete(col_names, [0, -1])
hist_tr = hist_tr.groupby('TRACT').sum()[col_names]

m2 = []
## Q2: CENRACE (63 cells)
for x in range(n4):     # race
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in hist_tr.columns if x in col[6:8]]
    m2.append(hist_tr[col_names].sum(axis=1).values)

# my noisy hist
filename_sf1 = 'data/franklin/microdata/franklin_hist_dp_tr.csv'
sf1 = pd.read_csv(filename_sf1)

m2_noisy = []
## Q2: CENRACE (63 cells)
for x in range(n4):     # race
    x = '{number:0{width}d}'.format(width=2, number=x)
    col_names = [col for col in sf1.columns if x in col[6:8]]
    m2_noisy.append(sf1[col_names].sum(axis=1).values)

noise = np.absolute(np.subtract(np.array(m2_noisy), np.array(m2)))

np.mean(noise)

1.4929577464788732

##### **Block group level**

Initialize bg-level noisy queries